In [1]:
import sqlalchemy
engine = sqlalchemy.create_engine("sqlite:///my_database.db", echo=True)   
    # Note: sqlite and not sqlite3.
    # If you want to use a different database for production (e.g. MySQL), just change the URL.
    # echo=True means log the SQL commands
engine

Engine(sqlite:///my_database.db)

## CREATE

In [3]:
# Instead of CREATE TABLE, we create a class that represents the table:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

# Define the "user" table:
class User(Base):
    __tablename__ = 'user'    
    # Define the columns in the user table:
    id = sqlalchemy.Column(sqlalchemy.Integer , primary_key= True)
    name = sqlalchemy.Column(sqlalchemy.String(20) , unique=False, nullable = False , default='Anonymous')
    phone = sqlalchemy.Column(sqlalchemy.Integer , unique = False)
    email = sqlalchemy.Column(sqlalchemy.String(20) , unique=True, nullable = False)
    password = sqlalchemy.Column(sqlalchemy.String(20), nullable= False)

    def __repr__(self):
        return f'User({self.id!r}, {self.name!r}, {self.phone!r}, {self.email!r})'

Base.metadata.create_all(engine)  # Create the database and all tables.

2022-05-11 11:14:48,410 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-11 11:14:48,411 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-05-11 11:14:48,411 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-11 11:14:48,413 INFO sqlalchemy.engine.Engine COMMIT


## INSERT

In [5]:
# Same data as before

name1 = 'Tom Pythonovitz'
phone1 = '3366858'
email1 = 'Tom.Pythonovitz@example.com'
# A very secure password
password1 = '12345'
 
name2 = 'Tammi Pythonovitz'
phone2 = '5557241'
email2 = 'Tammi@example.com'
password2 = 'Javansky'

name3 = 'George Rustniovsky'
phone3 = '33333'
email3 = 'GRust@example.com'
password3 = 'Rust for ever'

In [6]:
DBSession = sqlalchemy.orm.sessionmaker(bind=engine)
session = DBSession()

In [7]:
# Instead of INSERT INTO table, we create new objects:
user1 = User(name=name1, phone=phone1,  email=email1, password=password1)
session.add(user1)
print('First user inserted')

session.add(User(name=name2, phone=phone2,  email=email2, password=password2))
print('Second user inserted')

session.commit()

First user inserted
Second user inserted
2022-05-11 11:15:56,003 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-11 11:15:56,006 INFO sqlalchemy.engine.Engine INSERT INTO user (name, phone, email, password) VALUES (?, ?, ?, ?)
2022-05-11 11:15:56,007 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ('Tom Pythonovitz', '3366858', 'Tom.Pythonovitz@example.com', '12345')
2022-05-11 11:15:56,009 INFO sqlalchemy.engine.Engine INSERT INTO user (name, phone, email, password) VALUES (?, ?, ?, ?)
2022-05-11 11:15:56,010 INFO sqlalchemy.engine.Engine [cached since 0.003353s ago] ('Tammi Pythonovitz', '5557241', 'Tammi@example.com', 'Javansky')
2022-05-11 11:15:56,012 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
session.add(User(name=name3, phone=phone3,  email=email3, password=password3))
print('Third user inserted')
session.commit()

Third user inserted
2022-05-11 11:16:16,240 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-11 11:16:16,242 INFO sqlalchemy.engine.Engine INSERT INTO user (name, phone, email, password) VALUES (?, ?, ?, ?)
2022-05-11 11:16:16,243 INFO sqlalchemy.engine.Engine [cached since 20.24s ago] ('George Rustniovsky', '33333', 'GRust@example.com', 'Rust for ever')
2022-05-11 11:16:16,246 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
users = [User(name='Joee Javany', phone='2222', email='joo@example.mail', password='password'),
        User(name='Shirel Cplustik', phone='3333', email='cpp@example.mail', password='cppass'),
        User(name='Adam Kotlinberg', phone='4444', email='Adam_Kotlin@example.mail', password='JustPassword'),
        ]
# Insert many
session.bulk_save_objects(users)
print('Many users inserted')
session.commit()

2022-05-11 11:16:26,339 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-11 11:16:26,340 INFO sqlalchemy.engine.Engine INSERT INTO user (name, phone, email, password) VALUES (?, ?, ?, ?)
2022-05-11 11:16:26,341 INFO sqlalchemy.engine.Engine [generated in 0.00079s] (('Joee Javany', '2222', 'joo@example.mail', 'password'), ('Shirel Cplustik', '3333', 'cpp@example.mail', 'cppass'), ('Adam Kotlinberg', '4444', 'Adam_Kotlin@example.mail', 'JustPassword'))
Many users inserted
2022-05-11 11:16:26,344 INFO sqlalchemy.engine.Engine COMMIT


## SELECT

In [10]:
session.query(User).all()

2022-05-11 11:16:53,801 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-11 11:16:53,804 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.phone AS user_phone, user.email AS user_email, user.password AS user_password 
FROM user
2022-05-11 11:16:53,805 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()


[User(1, 'Tom Pythonovitz', 3366858, 'Tom.Pythonovitz@example.com'),
 User(2, 'Tammi Pythonovitz', 5557241, 'Tammi@example.com'),
 User(3, 'George Rustniovsky', 33333, 'GRust@example.com'),
 User(4, 'Joee Javany', 2222, 'joo@example.mail'),
 User(5, 'Shirel Cplustik', 3333, 'cpp@example.mail'),
 User(6, 'Adam Kotlinberg', 4444, 'Adam_Kotlin@example.mail')]

In [11]:
session.query(User).filter(User.name=="Tom Pythonovitz").all()

2022-05-11 11:17:04,293 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.phone AS user_phone, user.email AS user_email, user.password AS user_password 
FROM user 
WHERE user.name = ?
2022-05-11 11:17:04,293 INFO sqlalchemy.engine.Engine [generated in 0.00064s] ('Tom Pythonovitz',)


[User(1, 'Tom Pythonovitz', 3366858, 'Tom.Pythonovitz@example.com')]

In [12]:
tom = session.query(User).filter(User.name=="Tom Pythonovitz").first()
tom

2022-05-11 11:17:37,934 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.phone AS user_phone, user.email AS user_email, user.password AS user_password 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2022-05-11 11:17:37,935 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ('Tom Pythonovitz', 1, 0)


User(1, 'Tom Pythonovitz', 3366858, 'Tom.Pythonovitz@example.com')

In [13]:
tom.phone

3366858

## DROP

In [14]:
# To drop all tables:
Base.metadata.drop_all(engine) 

2022-05-11 11:18:00,630 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-11 11:18:00,631 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-05-11 11:18:00,631 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-11 11:18:00,634 INFO sqlalchemy.engine.Engine 
DROP TABLE user
2022-05-11 11:18:00,635 INFO sqlalchemy.engine.Engine [no key 0.00134s] ()
2022-05-11 11:18:00,642 INFO sqlalchemy.engine.Engine COMMIT


## close

In [36]:
session.close()

2022-04-26 21:36:22,970 INFO sqlalchemy.engine.Engine ROLLBACK
